# Imports

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

# Initialization

In [70]:
SCORE_DIR = "data/scores"

In [71]:
box_scores = os.listdir(SCORE_DIR)

In [72]:
len(box_scores)

3485

In [73]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

# Process 

## Parse HTML 

### Basic Advanced Stats Table Visualization

We want to remove the basic box score stats header and the duplicate stats header as they further complicate the parsing process


![Basic Advanced Stats](basic_advanced_stats.png)
![Basic Stats Table HTML](assets/basic_stats_table_html.png)


### Function

In [13]:
def parse_html(box_score):
    with open(box_score, "r") as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")] 
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

## Read Line Score Table

Pandas can process HTML table elements and turn them into dataframes. <br>
We only need the team name column and the final score column 
![Line Score Table](assets/line_score_table.png)

### Function

In [38]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team", "total"]]
    return line_score


## Read Stats from Table

Reference stats table picture above <br>
grab table based on the the team and type of stats and convert it into a Pandas Dataframe <br>
Handle int conversion and converting anything that is not a number such as "Did Not Play" into NaNs

In [47]:
def read_stats(soup, team, stat_type):
    df = pd.read_html(StringIO(str(soup)), attrs={"id": f"box-{team}-game-{stat_type}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df


## Read season Info

We would like to add more context to row of data by adding the season

In [55]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a.get("href") for a in nav.select("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In order to use data in a machine learning model we need to ensure all the neccessary data per game is in one row <br>
So we will grab the totals from each basic and advanced tables (the last row)

In [60]:
base_cols = None
games = [ ]

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = line_score.team.tolist()

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower() 
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [col for col in base_cols if "bpm" not in col]

        summary = summary[base_cols]
        summaries.append(summary)
    
    summary = pd.concat(summaries, axis=1).T
    
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game,game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]

    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 3485
200 / 3485
300 / 3485
400 / 3485
500 / 3485
600 / 3485
700 / 3485
800 / 3485
900 / 3485
1000 / 3485
1100 / 3485
1200 / 3485
1300 / 3485
1400 / 3485
1500 / 3485
1600 / 3485
1700 / 3485
1800 / 3485
1900 / 3485
2000 / 3485
2100 / 3485
2200 / 3485
2300 / 3485
2400 / 3485
2500 / 3485
2600 / 3485
2700 / 3485
2800 / 3485
2900 / 3485
3000 / 3485
3100 / 3485
3200 / 3485
3300 / 3485
3400 / 3485


In [62]:
games_df = pd.concat(games, ignore_index=True)

In [63]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,265.0,265.0,46.0,100.0,0.460,15.0,39.0,0.385,25.0,32.0,...,20.7,39.9,160.0,125.0,NOP,138,1,2020,2020-01-16,False
1,265.0,265.0,51.0,93.0,0.548,11.0,26.0,0.423,25.0,35.0,...,25.0,39.5,197.0,132.0,UTA,132,0,2020,2020-01-16,True
2,240.0,240.0,40.0,96.0,0.417,11.0,36.0,0.306,17.0,23.0,...,31.6,38.4,162.0,109.0,HOU,132,1,2020,2019-11-18,False
3,240.0,240.0,47.0,100.0,0.470,17.0,45.0,0.378,21.0,26.0,...,53.2,30.6,147.0,132.0,POR,108,0,2020,2019-11-18,True
4,240.0,240.0,48.0,91.0,0.527,20.0,41.0,0.488,13.0,21.0,...,50.0,33.5,147.0,141.0,TOR,105,1,2021,2021-03-03,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6965,240.0,240.0,38.0,82.0,0.463,12.0,37.0,0.324,15.0,18.0,...,60.0,29.8,159.0,115.0,MEM,112,0,2017,2017-02-13,False
6966,240.0,240.0,34.0,95.0,0.358,7.0,33.0,0.212,17.0,23.0,...,59.2,34.5,231.0,103.0,LAC,108,1,2019,2018-10-19,False
6967,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
6968,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [68]:
games_df.to_csv("nba_games.csv")